In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys

sys.path.append("/Users/ruhana/Agent-E/test")
from load_annotations import AnnotationLoader

None


In [2]:
LOG_PATH = "/Users/ruhana/Agent-E/new_ruhana_notes/All/logs/test_results_for_full_text/"
RESULT_PATH = "/Users/ruhana/Agent-E/new_ruhana_notes/All/results/results_for_test_full_text/"
self_refine_annotations = AnnotationLoader(LOG_PATH, RESULT_PATH)

# Load final results as a dataframe
self_refine_results = self_refine_annotations.get_all_annotation_results()
self_refine_results.columns

Index(['task_id', 'task_index', 'start_url', 'intent', 'last_url', 'tct',
       'start_ts', 'completion_ts', 'last_statement',
       'validator_last_statement', 'score', 'reason', 'validate_score',
       'validate_reason', 'screenshot_rate', 'validation_count',
       'total_main_chat_token', 'main_chat_length', 'total_trajectory_length'],
      dtype='object')

In [14]:
# all_self_refinement_results = {}

# for mode in ["text", "vision", "text_vision"]:
#     LOG_PATH = f"/Users/ruhana/Agent-E/new_ruhana_notes/All/logs/test_results_for_full_{mode}/"
#     RESULT_PATH = f"/Users/ruhana/Agent-E/new_ruhana_notes/All/results/results_for_test_full_{mode}/"
#     annon_loader = AnnotationLoader(LOG_PATH, RESULT_PATH)
#     all_self_refinement_results[mode] = annon_loader.get_all_annotation_results()
#     print(len(all_self_refinement_results[mode]))

In [4]:
LOG_PATH = "/Users/ruhana/Agent-E/ruhana_notes/baseline_annotated/original_annotations/All/logs"
RESULT_PATH = "/Users/ruhana/Agent-E/ruhana_notes/baseline_annotated/original_annotations/All/results"
original_annotation = AnnotationLoader(LOG_PATH, RESULT_PATH)

# Load final results as a dataframe
original_results = original_annotation.get_all_annotation_results()
original_results.columns

# make sure original matches the same subset!!!
self_refine_task_id = list(self_refine_results['task_id'])
original_results = original_results[original_results['task_id'].isin(self_refine_task_id)]

In [5]:
original_subset = original_results[['tct',
       'score', 'compute_cost.cost', 'compute_cost.prompt_tokens',
       'compute_cost.completion_tokens', 'compute_cost.total_tokens',
       'total_main_chat_token', 'total_trajectory_length', 'main_chat_length']]
correlation_matrix = original_subset.corr()

# # Plotting the correlation matrix using matplotlib
# plt.figure(figsize=(6, 5))
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".1f")
# plt.title("Cost, Task Difficulty Correlation Matrix")
# plt.show()

In [6]:
# View mean score and group by start_url

original_subset = original_results[['start_url', 'tct',
       'score', 'compute_cost.cost', 'compute_cost.prompt_tokens',
       'compute_cost.completion_tokens', 'compute_cost.total_tokens',
       'total_main_chat_token', 'total_trajectory_length', 'main_chat_length']]
original_by_url = original_subset.groupby('start_url').mean()

self_refine_subset = self_refine_results[['start_url', 'tct',
       'score','total_main_chat_token', 'total_trajectory_length', 'main_chat_length']]
self_refine_by_url = self_refine_subset.groupby('start_url').mean()

print(f"Agent-E Acc: {original_results['score'].mean()}\nSelf-Refine: {self_refine_subset['score'].mean()}")

Agent-E Acc: 0.7474489795918368
Self-Refine: 0.8137755102040817


In [7]:
# View scores by URL
# self_refine_by_url
# original_by_url

In [8]:
original_wrong = set(original_results[original_results['score'] == 0.0]['task_id'])
original_correct = set(original_results[original_results['score'] == 1.0]['task_id'])
refiner_correct = set(self_refine_results[self_refine_results['score'] == 1.0]['task_id'])
refiner_wrong = set(self_refine_results[self_refine_results['score'] == 0.0]['task_id'])

improved_tasks = list(refiner_correct - original_correct)
print(f"{len(improved_tasks)}/{len(refiner_wrong) + len(refiner_correct)}")

improved_tasks_df = original_results[original_results['task_id'].isin(improved_tasks)].sort_values(by='task_id', ascending=True)
url_counts = improved_tasks_df['start_url'].value_counts()
url_counts

64/392


start_url
https://www.google.com/travel/flights/    9
https://www.amazon.com/                   7
https://www.apple.com/                    7
https://arxiv.org/                        7
https://www.allrecipes.com/               6
https://www.bbc.com/news/                 5
https://www.booking.com/                  5
https://github.com/                       5
https://dictionary.cambridge.org/         4
https://www.espn.com/                     3
https://www.coursera.org/                 2
https://huggingface.co/                   2
https://www.google.com/maps/              1
https://www.google.com/                   1
Name: count, dtype: int64

In [12]:
#improved_tasks_df
improved_df_refine = self_refine_results[self_refine_results['task_id'].isin(improved_tasks)].sort_values(by='task_id', ascending=True)
improved_df_and_validation = self_refine_results[self_refine_results['validation_count'] > 1].sort_values(by='task_id', ascending=True)

In [13]:
improved_df_original = original_results[original_results['task_id'].isin(improved_tasks)].sort_values(by='task_id', ascending=True)
improved_df_original[['start_url', 'intent','reason', 'score']]

,start_url,intent,reason,score
0,https://www.allrecipes.com/,Find a recipe for a vegetarian lasagna that ha...,All these recipes do not explicitly include zu...,0.0
0,https://www.allrecipes.com/,Find a recipe for a vegetarian lasagna under 6...,After reviewing the first three vegetarian las...,0.0
0,https://www.allrecipes.com/,Search for a recipe for Beef Wellington on All...,none of the recipes meet the specified criteri...,0.0
0,https://www.allrecipes.com/,Find a recipe for a vegetarian lasagna that ha...,The page does not directly address the initial...,0.0
0,https://www.allrecipes.com/,Find a popular quinoa salad recipe on Allrecip...,The search result doesn't meet the requirement...,0.0
...,...,...,...,...
0,https://www.google.com/travel/flights/,Search for round-trip flights from Helsinki to...,unable to filter the specified criteria,0.0
0,https://www.google.com/maps/,"Find all Uniqlo locations in Chicago, IL.","No requested webpage found, error",0.0
0,https://www.google.com/,"Show me a list of comedy movies, sorted by use...",no actual results found,0.0
0,https://huggingface.co/,Look up TRL's forward modelling in the hugging...,Agent answer is empty,0.0


In [ ]:
from openai import OpenAI
client = OpenAI()

data = improved_df_original[['start_url', 'intent','reason']].to_csv()
question = f"I need help analyzing my results from an experiment. In this experiment, I had a web-navigation agent accomplish many tasks on live websites. Each failure case was annotated by a real human with the reason for the web-navigator's failure. Help me analyze the failure cases. Please categorize the top-5 failure cases and give me examples of each. Think deeply about why each of these cases may occur and what causes each error type. Below is a csv file containing failure cases only. There are three columns: 1) start_url, which is the website the task was performed on 2) the intent, this is the task that the agent was given to perform and 3) the reason, this is the annotators' explaination for why the task failed: {data}"

completion = client.chat.completions.create(
            model="gpt-4",  # Specify the model (e.g., gpt-4 or gpt-3.5-turbo)
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": question},
            ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Based on the data provided, the top 5 categories of failure cases can be grouped mainly into: \n\n1) **Insufficient Matching Criteria**\n \nExamples:\n- "Find a recipe for a vegetarian lasagna that has at least a four-star rating and uses zucchini."\n   \n- "Find a Blue iPhone 12 Pro 128gb and add to cart."\n\nReason: This happens when the agent couldn\'t find the requested information because the criteria were too specific or the task was beyond the capabilities of the agent. The agent requires enhancements in terms of advanced functionalities.\n\n2) **Failure in Filtering/Sorting/Comparing Data**\n\nExamples:\n- "Find a stainless steel kitchen sink with double bowls on Amazon. Sort the results and find the cheapest one with FREE delivery."\n\n- "Compare the prices and chips for the iPhone 14 Pro and iPhone 15 Pro models directly from Apple\'s website."\n\nReason: This error occurs when the agent fails to filter, sort, or compare data accurately due to t

In [ ]:
def print_confusion(gpt_eval):
    true_positive = gpt_eval[(gpt_eval['pred_score'] == 1) & (gpt_eval['score'] == 1)]
    true_negative = gpt_eval[(gpt_eval['pred_score'] == 0) & (gpt_eval['score'] == 0)]
    false_positive = gpt_eval[(gpt_eval['pred_score'] == 1) & (gpt_eval['score'] == 0)]
    false_negative = gpt_eval[(gpt_eval['pred_score'] == 0) & (gpt_eval['score'] == 1)]
    print(len(true_positive), len(true_negative))
    print(len(false_positive), len(false_negative))
    gpt_eval['validator_correct'] = gpt_eval['score'] == gpt_eval['pred_score']
    print("Accuracy:", gpt_eval['validator_correct'].mean())
    return

In [18]:
from validation_agent.analyze_failure import *

model = "gpt-4o"
task_id = 0
intent = original_annotation.get_intent(task_id)
main_chat_sequence = original_annotation.get_high_level_trajectory(task_id)
screenshot_paths = original_annotation.get_screenshot_paths(task_id)

analysis = analyze_failure_text(main_chat_sequence, intent, model=model)

NameError: name 'model' is not defined

In [ ]:
# Store all of them into a dataframe (i.e. loop from improved_tasks)
# Then summarize from a high-level (i.e. call openai for another step of anlaysis)

In [ ]:
# TODO: go back and add ALL organized data...